# db_plots

### This notebook queries the databases to generate a matrix of plots for the number of articles/images per year, run separately for each category.

#### Setup

Import required libraries, connect to SQLite database, create cursor, fetch table info

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import sqlite3

In [ ]:
# path on rte
db_path = "/home/rte/data/db/arxiv_db_images.sqlite3"

In [ ]:
# import the sqlite3 database and create a cursor

db = sqlite3.connect(db_path)
c = db.cursor()

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("metadata"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

In [ ]:
c.execute('PRAGMA TABLE_INFO({})'.format("images"))
info = c.fetchall()

print("\nColumn Info:\nID, Name, Type, NotNull, DefaultVal, PrimaryKey")
for col in info:
    print(col)

#### Build category list
First get a full list of all the primary categories by querying the SQLite database.

In [ ]:
# list primary categories by articles

c.execute('''
    SELECT substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1), count(metadata.identifier)
    FROM metadata
    GROUP BY substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1)
    ORDER BY count(metadata.identifier) DESC    
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# list primary categories by associated images

c.execute('''
    SELECT substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1), count(images.identifier)
    FROM images
    LEFT JOIN metadata ON images.identifier = metadata.identifier 
    GROUP BY substr(trim(metadata.cat),1,instr(trim(metadata.cat)||' ',' ')-1)
    ORDER BY count(images.identifier) DESC    
    ''')
rows = c.fetchall()
for row in rows:
    print(row)

In [ ]:
print(len(rows))

In [ ]:
# store the list of categories in catlist

catlist = rows
for cat in catlist:
    print(str(cat[0]))

In [ ]:
# testing that both SQL queries give the same list of categories
# the lists seem to be slightly different in order but have the same number of categories

catlist_images = rows
print("are both lists the same?")
print(catlist_images == catlist)
print("do both lists have the same number of elements?")
print(len(catlist_images) == len(catlist))
print(len(catlist))

#### Query DB for data
Then use that list of primary categories to query the db for how many articles per year. Store it in the `data` variable. The first command is the number of articles only so only requires the `metadata` table. The second block below searches by number of images so also requires the `images` table in the SQLite database.

In [ ]:
# total number of articles for each year by category

sql = ('''
    SELECT count(metadata.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM metadata
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = ?
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')

data = []

for cat in catlist:
    print("querying for category: " + str(cat[0]))
    c.execute(sql, (cat[0], ))
    rows = c.fetchall()
    
#     print("total number of images found: " + str(len(rows)))
    print(rows)
#     print("total number of articles: " + rows[0][0])

    years = []
    totals = []
    
    for row in rows:
        years.append(row[1])
        totals.append(row[0])
        
    newdata = [cat[0], years, totals]
    data.append(newdata)

print("*" * 20)
print("done")

In [ ]:
# total number of images for each year by category

sql = ('''
    SELECT count(images.identifier), strftime("%Y", metadata.created) as 'Y'
    FROM images
    LEFT JOIN metadata on images.identifier = metadata.identifier
    WHERE substr(trim(cat),1,instr(trim(cat)||' ',' ')-1) = ?
    GROUP BY strftime("%Y", metadata.created)
    ORDER BY strftime("%Y", metadata.created) ASC
    ''')

data = []

for cat in catlist:
    print("querying for category: " + str(cat[0]))
    c.execute(sql, (cat[0], ))
    rows = c.fetchall()
    
#     print("total number of images found: " + str(len(rows)))
    print(rows)
#     print("total number of articles: " + rows[0][0])

    years = []
    totals = []
    
    for row in rows:
        years.append(row[1])
        totals.append(row[0])
        
    newdata = [cat[0], years, totals]
    data.append(newdata)

print("*" * 20)
print("done")

#### Clean data
- remove any entries of "2019" in the years and articles columns of data (don't have full data for this year).
- rewrite all entries as integers rather than strings (otherwise there will be problems when adjusting the axes)
- find the minimum and maximum for any entries, so that we can set our axes later as needed.

In [ ]:
# if there is "2019" in the list of years, get the index and remove from both the year and no. article lists

for cat in data:
    while "2019" in cat[1]:
        index = cat[1].index("2019")
        print(cat[0])
        print(index)
        del cat[2][index]
        del cat[1][index]
        print("*" * 20)

In [ ]:
# test to make sure there is still a total for each year

for cat in data:
    if len(cat[1]) != len(cat[2]):
        print("problem with category: " + cat)

In [ ]:
# fix year entries appearing as string by going through each and re-writing as integer

for cat in data:
    for k in enumerate(cat[1]):
        print(cat[1][k[0]])
        cat[1][k[0]] = int(cat[1][k[0]])

In [ ]:
print(data)

In [ ]:
# get the maximums and minimums of year and no. articles for figuring out axes

import math

minY = math.inf
maxY = -(math.inf)
minA = math.inf
maxA = -(math.inf)

for cat in data:
#     print(len(cat))
    print(cat[0])
    if min(cat[1]) < minY: minY = min(cat[1])
    if max(cat[1]) > maxY: maxY = max(cat[1])
    if min(cat[2]) < minA: minA = min(cat[2])
    if max(cat[2]) > maxA: maxA = max(cat[2])
    print("min year: " + str(min(cat[1])))
    print("max year: " + str(max(cat[1])))
    print("min articles/images: " + str(min(cat[2])))
    print("max articles/images: " + str(max(cat[2])))
    print("*" * 20)
    
print("minY: " + str(minY))
print("maxY: " + str(maxY))
print("minA: " + str(minA))
print("maxA: " + str(maxA))
              
print("done")

#### Save data 
Interim progress, to prevent having to run SQL queries again : )
Save as either json file or pickle for reloading

#### json
saves as human-readable JSON format

In [ ]:
import json

In [ ]:
filename = "articles_cat_year.json" 

In [ ]:
filename = "images_cat_year.json"

In [ ]:
with open(filename, "w") as write_file:
    json.dump(data, write_file)
    write_file.close()

In [ ]:
load_json = []

with open(filename, "r") as read_file:
    load_json = json.load(read_file)
    read_file.close()

In [ ]:
print(load_json)

In [ ]:
data == loadedjson

#### pickle
Save data as a serialized file using pickle

In [ ]:
import pickle

In [ ]:
filename = "images_cat_year.pickle"

In [ ]:
filename = "articles_cat_year_clean.pickle"

In [ ]:
with open(filename, "wb") as write_file:
    pickle.dump(data, write_file)
    write_file.close()

In [ ]:
load_data = []

with open(filename, "rb") as read_file:
    load_data = pickle.load(read_file)
    read_file.close()

Load in the imported data in the `data` variable

In [ ]:
data = load_data

Testing loaded data

In [ ]:
print(load_data == data)

In [ ]:
print(load_data)

In [ ]:
print(data)

#### Save data in org format
Use org-friendly table format. This can be printed to console or written to a file. For posting to Github and rendered in Github markdown.

In [ ]:
# write the data in an org-friendly format for posting on github
for cat in data:
    print("* " + cat[0])
    joined = list(zip(cat[1], cat[2]))
    #     print(joined)
    print("|-|-|")
    for j in joined:
        print('|' + str(j[0]) + "|" + str(j[1]) + "|")
    print("|-|-|")

In [ ]:
# write the data to a file
with open("stats_images_cat_year.org", "w") as write_file:
    for cat in data:
        print("* " + cat[0], file=write_file)
        joined = list(zip(cat[1], cat[2]))
        #     print(joined)
        print("|-|-|", file=write_file)
        for j in joined:
            print('|' + str(j[0]) + "|" + str(j[1]) + "|", file=write_file)
        print("|-|-|", file=write_file)
write_file.close()

#### Plotting matrix of scatterplots

Plot data in two formats
- with shared x and y axes, for comparison across data
- with individual x and y axes taken from min/max of each plot automatically, for individual trends
- finally, save as high resolution (300 dpi) image

In [ ]:
# plot figures with shared x and y axes using the min/max year/article numbers from the cleaning step

xdim = 15
ydim = 12

fig, ax = plt.subplots(ydim, xdim, sharex='col', sharey='row')
fig.set_size_inches(40, 30)

data_size = len(data)

for i in range(ydim):
    for j in range(xdim):
        idx = (i * xdim) + j
        if idx < data_size:
#             ax[i, j].plot(data[idx][1], data[idx][2], '--o')
            ax[i, j].plot(data[idx][1], data[idx][2], '--r.')
            ax[i, j].title.set_text(data[idx][0])
            # add one to the maximum year for alignment
            ax[i, j].axis([minY, maxY+1, minA, maxA])

In [ ]:
# fig.savefig("plot_articles_cat_year_04.png", dpi=300)
fig.savefig("plot_images_cat_year_03.png", dpi=300)

In [ ]:
xdim = 15
ydim = 12

fig, ax = plt.subplots(ydim, xdim)
fig.subplots_adjust(hspace=0.4, wspace=0.4)
fig.set_size_inches(40, 30)

data_size = len(data)

for i in range(ydim):
    for j in range(xdim):
        idx = (i * xdim) + j
        if idx < data_size:
#             ax[i, j].plot(data[idx][1], data[idx][2], '--o')
            ax[i, j].plot(data[idx][1], data[idx][2], '--r.')
            ax[i, j].title.set_text(data[idx][0])
#             ax[i, j].axis([minY, maxY+1, minA, maxA])

In [ ]:
# fig.savefig("plot_articles_cat_year_indax_01.png", dpi=300)
fig.savefig("plot_images_cat_year_indax_03.png", dpi=300)